# Import the libraries

In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from sklearn.preprocessing import normalize


In [ ]:
# cv2: Used for computer vision tasks such as face detection and face recognition.
# numpy: Provides support for working with arrays.
# tkinter: Standard Python library to build graphical user interfaces.
# PIL (Python Imaging Library): Used for opening, manipulating, and converting image formats, particularly useful in conjunction with Tkinter.
# sklearn.preprocessing.normalize: Used to normalize face images (although not used in the current version).

# Initialize Known Faces and Labels

In [3]:
uploaded_images = ['ref1.jpg','ref2.jpg','r3.jpg']
known_face_labels = []
known_face_images = []
mnx=["Elon musk","mark zuckerberg","Sunder pichai"]
confidence_threshold = 50

In [4]:
# uploaded_images: List of image file names of the people you want to recognize.
# known_face_labels: Empty list to store labels (used to identify which person the face belongs to).
# known_face_images: Empty list to store cropped face images.
# mnx: List of corresponding names for each face in uploaded_images.
# confidence_threshold: Threshold for face recognition. Lowering the value makes the recognition stricter (lower tolerance for mismatches)

# Initialize Face Cascade for Detection

In [5]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [6]:
# CascadeClassifier: Loads the pre-trained Haar cascade for frontal face detection, which is provided by OpenCV.

# Preprocess the Face Image


In [7]:
def preprocess_image(image):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.equalizeHist(image)
    return image


In [8]:
# Convert to grayscale: Converts the image to grayscale if it’s in color.
# Histogram Equalization: Adjusts the contrast of the grayscale image to improve the face detection and recognition accuracy.

# Load Images and Train the Face Recognizer

In [9]:
def load_images_and_train():
    global face_recognizer
    for idx, img_path in enumerate(uploaded_images):
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Error: Could not load image {img_path}")
            continue
        image = preprocess_image(image)
        faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        for (x, y, w, h) in faces:
            face = image[y:y+h, x:x+w]
            known_face_images.append(face)
            known_face_labels.append(idx)
    if not known_face_images:
        messagebox.showerror("Error", "No faces found in uploaded images.")
        return
    
    face_recognizer.train(known_face_images, np.array(known_face_labels))


In [10]:
# The function loops through the uploaded_images, converts them to grayscale, and detects faces using detectMultiScale().
# Each detected face is cropped and stored in known_face_images.
# The face label (index from mnx) is stored in known_face_labels.
# After processing, the function uses face_recognizer.train() to train the recognizer on the labeled images.

# Start Camera and Recognize Faces

In [11]:
def start_camera():
    global cap
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        messagebox.showerror("Error", "Could not open camera.")
        return
    
    def update_frame():
        ret, frame = cap.read()
        if not ret:
            messagebox.showerror("Error", "Could not read frame.")
            return
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for widget in faces_frame.winfo_children():
            widget.destroy()

        for (x, y, w, h) in faces:
            face = gray_frame[y:y+h, x:x+w]
            face = preprocess_image(face)
            label, confidence = face_recognizer.predict(face)
            if confidence < confidence_threshold:
                name = mnx[label]
                cv2.putText(frame, f"{name} ({round(confidence, 2)})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Not Matched", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            detected_face = frame[y:y+h, x:x+w]
            detected_face = cv2.resize(detected_face, (100, 100))
            detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2RGB)
            face_img = Image.fromarray(detected_face)
            face_imgtk = ImageTk.PhotoImage(image=face_img)
            face_label = tk.Label(faces_frame, image=face_imgtk)
            face_label.image = face_imgtk
            face_label.pack(side='left', padx=10)

        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        camera_label.imgtk = imgtk
        camera_label.configure(image=imgtk)
        camera_label.after(10, update_frame)

    update_frame()


In [12]:
# Open Camera: Attempts to open the camera feed using VideoCapture(0).
# Process Each Frame: Converts each frame to grayscale, detects faces, and matches them using the face recognizer.
# Display Faces and Names: Detected faces are drawn on the frame with a rectangle, and the name of the person is written if recognized.
# Tkinter Update: The update_frame function continuously updates the Tkinter GUI with the live video feed.

# Stop Camera

In [13]:
def stop_camera():
    global cap
    cap.release()
    cv2.destroyAllWindows()
    camera_label.config(image='')
    for widget in faces_frame.winfo_children():
        widget.destroy()


In [14]:
# Releases the camera and clears the video feed displayed in the GUI.

# Upload a New Image

In [16]:
def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        new_image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if new_image is None:
            messagebox.showerror("Error", "Invalid image format")
            return
        
        new_image = preprocess_image(new_image)
        faces = face_cascade.detectMultiScale(new_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        if len(faces) == 0:
            messagebox.showerror("Error", "No face detected in the image.")
            return
        
        for (x, y, w, h) in faces:
            face = new_image[y:y+h, x:x+w]
            known_face_images.append(face)
            known_face_labels.append(len(mnx))
            mnx.append(f"Person {len(mnx) + 1}")
        face_recognizer.train(known_face_images, np.array(known_face_labels))
        messagebox.showinfo("Success", "New face added and recognizer updated.")


In [17]:
# Allows users to add new faces to the recognizer by uploading an image. The new face is detected, processed, and added to the list of known faces.

# GUI Setup

In [19]:
root = tk.Tk()
root.title("Face Recognition")
root.geometry("1000x700")
root.configure(bg='#f0f0f0')

camera_label = tk.Label(root, text="Camera Feed", bg='#f0f0f0', fg='#333', font=('Arial', 12))
camera_label.pack(pady=10)

faces_frame = tk.Frame(root, bg='#f0f0f0')
faces_frame.pack(pady=10)

control_frame = tk.Frame(root, bg='#f0f0f0')
control_frame.pack(pady=20)

start_button = tk.Button(control_frame, text="Start Camera", command=start_camera, bg='#4CAF50', fg='white', font=('Arial', 12), padx=10, pady=5)
start_button.grid(row=0, column=0, padx=10)

stop_button = tk.Button(control_frame, text="Stop Camera", command=stop_camera, bg='#f44336', fg='white', font=('Arial', 12), padx=10, pady=5)
stop_button.grid(row=0, column=1, padx=10)

upload_button = tk.Button(control_frame, text="Upload Image", command=upload_image, bg='#2196F3', fg='white', font=('Arial', 12), padx=10, pady=5)
upload_button.grid(row=0, column=2, padx=10)
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
load_images_and_train()

root.mainloop()

Error: Could not load image ref2.jpg
